## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-19-20-47-45 +0000,nyt,Shell Detonated Over Interstate 5 During Marin...,https://www.nytimes.com/2025/10/19/us/politics...
1,2025-10-19-20-45-52 +0000,nypost,Zohran Mamdani’s imam ally Siraj Wahhaj has hi...,https://nypost.com/2025/10/19/us-news/zohran-m...
2,2025-10-19-20-42-20 +0000,nypost,United Airlines pilot injured after mystery ob...,https://nypost.com/2025/10/19/us-news/united-a...
3,2025-10-19-20-39-45 +0000,startribune,"Takeaways: Jalen Hurts’ big plays, Carson Went...",https://www.startribune.com/minnesota-vikings-...
4,2025-10-19-20-39-28 +0000,nypost,Top White House official says Butler assassina...,https://nypost.com/2025/10/19/us-news/wh-big-s...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2382/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
47,trump,30
111,no,16
112,kings,13
186,gaza,11
182,israel,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
16,2025-10-19-20-07-07 +0000,nyt,‘No Kings’ Protests Against Trump Draws Thousa...,https://www.nytimes.com/2025/10/18/us/protests...,83
52,2025-10-19-17-32-48 +0000,nyt,Vance Hosts Marines Demonstration Amid ‘No Kin...,https://www.nytimes.com/2025/10/19/us/politics...,83
85,2025-10-19-12-33-49 +0000,nypost,Trump posts wild AI video showing him flying f...,https://nypost.com/2025/10/19/us-news/trump-po...,82
127,2025-10-19-06-11-20 +0000,bbc,Millions turned out for anti-Trump 'No Kings' ...,https://www.bbc.com/news/articles/c93xgyp1zv4o...,82
136,2025-10-19-03-33-45 +0000,missionlocal,"Photos: Defiance, solidarity fill S.F. streets...",https://missionlocal.org/2025/10/photos-defian...,77


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
16,83,2025-10-19-20-07-07 +0000,nyt,‘No Kings’ Protests Against Trump Draws Thousa...,https://www.nytimes.com/2025/10/18/us/protests...
31,46,2025-10-19-19-17-32 +0000,nyt,"Israel Strikes Gaza and Temporarily Halts Aid,...",https://www.nytimes.com/2025/10/19/world/middl...
32,40,2025-10-19-19-13-36 +0000,nypost,House Dem leader Hakeem Jeffries has awkward p...,https://nypost.com/2025/10/19/us-news/hakeem-j...
106,40,2025-10-19-09-57-15 +0000,nypost,"Scooter-riding thieves break into Louvre, swip...",https://nypost.com/2025/10/19/world-news/louvr...
126,36,2025-10-19-06-31-02 +0000,bbc,Afghanistan pulls out of cricket series after ...,https://www.bbc.com/news/articles/c20pnz01x0eo...
80,29,2025-10-19-13-21-31 +0000,nypost,Trump says he’s cutting off all subsidies to C...,https://nypost.com/2025/10/19/us-news/trump-cu...
51,28,2025-10-19-17-38-31 +0000,nypost,New Jersey GOP sounds alarms about ballot secu...,https://nypost.com/2025/10/19/us-news/nj-gop-s...
78,26,2025-10-19-13-53-46 +0000,nypost,Lying ex-Rep. George Santos evokes ‘Jesus Chri...,https://nypost.com/2025/10/19/us-news/ex-rep-g...
20,21,2025-10-19-19-52-03 +0000,nypost,Andrew Cuomo touts Jessica Tisch as NYPD commi...,https://nypost.com/2025/10/19/us-news/andrew-c...
26,21,2025-10-19-19-27-11 +0000,nypost,Secret Service investigating possible sniper’s...,https://nypost.com/2025/10/19/us-news/secret-s...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
